# Final Project: Euchre
Tyler Gibson & Carter Shavitz | CSC 5661

https://en.wikipedia.org/wiki/Euchre

https://cardgames.io/euchre/

In [1]:
import random
from player import Player
from score import Score

import numpy as np
import re
import time

## Euchre Environment

In [2]:
class Euchre:
    """
    A class to manage the euchre board.
    """

    def __init__(self, config: dict) -> None:
        """
        The constructor for the class.

        Parameters
        ----------
        config : dict
            The configuration parameters for euchre.
        """
        self.config = config
        self.score_to_win = self.config['score_to_win']  # 5, 7, or 10
        self.values = self.config['values']  # 9 - A that can include 7's and/or 8's

        self.num_players = 4
        self.suits = ['♠', '♥', '♦', '♣']
        self.deck = [f"{value}{suit}" for suit in self.suits for value in self.values]

        self.players = [Player(id=id + 1, team_id=1 if id % 2 == 0 else 2) for id in range(self.num_players)]
        self.score = Score()

        self.team_1_score = 0
        self.team_2_score = 0
        
        self.top_card = None
        self.trump = None
        self.dealer = 1

    def deal_cards(self, distribution_pattern=(3, 2)):
        """
        Deal the cards for a new 

        Parameters
        ----------
        

        Returns
        -------

        """
        random.shuffle(self.deck)
    
        players_hands = {
            '1': [],
            '2': [],
            '3': [],
            '4': []
        }

        deck_index = 0
        for count, player in enumerate(list(players_hands.keys()) * 2):
            if (count % 2 == 0 and count < len(players_hands)) or (count % 2 == 1 and count >= len(players_hands)):
                players_hands[player].append(self.deck[deck_index:deck_index+distribution_pattern[0]])
                deck_index += distribution_pattern[0] 
            else:
                players_hands[player].append(self.deck[deck_index:deck_index+distribution_pattern[1]])
                deck_index += distribution_pattern[1]

        for key in players_hands.keys():
            players_hands[key] =  [item for sublist in players_hands[key] for item in sublist]
                    
        return players_hands, self.deck[deck_index]
    
    def reset_game(self):
        """
        Reset the game.
        """
        self.players = [Player(id=id + 1, team_id=1 if id % 2 == 0 else 2) for id in range(self.num_players)]
        
        self.team_1_score = 0
        self.team_2_score = 0

        self.top_card = None
        self.trump = None

        self.dealer = np.random.choice(range(self.num_players)) + 1

        self.reset_hand()

    def reset_hand(self):
        """
        Reset the hand.
        """
        for player in self.players:
            player.points = 0
            
        players_hands, top_card = self.deal_cards(distribution_pattern=(3, 2))

        for count, player in enumerate(self.players):
            player.set_hand(players_hands[str(count + 1)])

        self.top_card = top_card

        self.reset_trick()

    def reset_trick(self):
        """
        Reset the trick.
        """
        self.dealer = 1 if self.dealer == 4 else self.dealer + 1  

    def reward(self, won_trick, teammate_won) -> float:
        """
        TODO

        Returns
        -------
        float
            The reward based on the action.
        """
        return 1

    def step(self, a) -> dict:
        """
        TODO

        Parameters
        ----------
        a : TODO
            TODO

        Returns
        -------
        dict
            TODO
        """
        if a == "order_it_up":
            pass
        elif a == "pass":
            pass


        return {

        }

In [3]:
env_config = {
    'score_to_win': 10,
    'values': ['9','10','J','Q','K','A']
}

euchre = Euchre(env_config)

score = Score()

In [4]:
euchre.reset_hand()

for player in euchre.players:
    print(player.hand, player.team_id)

euchre.top_card, euchre.dealer

['9♣', 'A♣', 'K♣', '9♠', 'K♠'] 1
['J♥', 'A♥', '9♥', 'Q♠', 'Q♣'] 2
['A♦', '10♣', 'J♦', 'A♠', 'Q♥'] 1
['K♥', '10♦', 'J♣', 'Q♦', 'K♦'] 2


('10♠', 2)

In [28]:
euchre.reset_game()

hands = 1  # keep track of which game we are on
while euchre.team_1_score < euchre.score_to_win and euchre.team_2_score < euchre.score_to_win:  # while game not complete
    current_player_id = 1 if euchre.dealer == 4 else euchre.dealer + 1  # get the player to the left of the dealer to start the hand
    player_choose_trump = [player for player in euchre.players if player.id == current_player_id][0]  # assuming the player to the left of dealer chooses trump and starts (not based off of actual rules)
    player_choose_trump.trick_team = 'makers'  # assign the first player as the Makers

    # loop through 5 tricks (each player should have 5 cards)
    for _ in range(5):
        lead_suit = ''  # init lead suit

        # loop through each player for their turn
        for x in range(euchre.num_players):
            current_player = [player for player in euchre.players if player.id == current_player_id][0]  # get current player based on id
            current_player.current_card = current_player.choose_card(random=True)  # set the current card of the player

            # if it is the first player, determine the lead suit
            if x == 0:
                lead_suit = re.findall(score.suits, current_player.current_card)[0] # use regex

            current_player_id = 1 if current_player_id == 4 else current_player_id + 1  # update the current player id to the next player

        trump_suit =  re.findall(score.suits, euchre.top_card)[0]  # determine the trump suit

        # find the player id who won the trick
        highest_player_id = score.score_trick(
            players=euchre.players,
            trump_suit=trump_suit,
            lead_suit=lead_suit
        )
        highest_player = [player for player in euchre.players if player.id == highest_player_id][0]  # get player with won trick
        highest_player.points += 1  # update their points by 1 for the hand
        current_player_id = highest_player_id  # the player who won the trick will lead the next trick

    hand_scores = score.score_hand(euchre.players, solo_call=False)  # calculate the hand scores by looking at each players points for all tricks
    
    # update team scores
    if player_choose_trump.team_id == 1:
        euchre.team_1_score += hand_scores['makers']
        euchre.team_2_score += hand_scores['defenders']
    else:
        euchre.team_2_score += hand_scores['makers']
        euchre.team_1_score += hand_scores['defenders']

    print(f'Hand {hands}\t\t Team 1 Score: {euchre.team_1_score}\t\tTeam 2 Score: {euchre.team_2_score}')
    hands += 1  # update hand counter
    euchre.reset_hand()  # reset hand

print('Team 1 Wins' if euchre.team_1_score >= 10 else 'Team 2 Wins')

Hand 1		 Team 1 Score: 0		Team 2 Score: 2
Hand 2		 Team 1 Score: 2		Team 2 Score: 2
Hand 3		 Team 1 Score: 3		Team 2 Score: 2
Hand 4		 Team 1 Score: 3		Team 2 Score: 4
Hand 5		 Team 1 Score: 5		Team 2 Score: 4
Hand 6		 Team 1 Score: 5		Team 2 Score: 6
Hand 7		 Team 1 Score: 7		Team 2 Score: 6
Hand 8		 Team 1 Score: 7		Team 2 Score: 8
Hand 9		 Team 1 Score: 9		Team 2 Score: 8
Hand 10		 Team 1 Score: 9		Team 2 Score: 10
Team 2 Wins
